<a href="https://colab.research.google.com/github/saimasharleen/Book-Recommendation-System/blob/main/lastdatamining_withrandomdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
pip install faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import os
import csv
from faker import Faker

fake = Faker()

with open('random_books.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13', 'language_code', 'num_pages', 'ratings_count', 'text_reviews_count', 'publication_date', 'publisher'])
    for i in range(20000):
        writer.writerow([i, fake.catch_phrase(), fake.name(), fake.random_number(digits=2), fake.isbn10(separator="-"), fake.isbn13(separator="-"), fake.language_code(), fake.random_number(digits=3), fake.random_number(digits=5), fake.random_number(digits=3), fake.date_this_decade(), fake.company()])

os.rename('random_books.csv', 'book_dataset.csv')


In [19]:
# importing the csv library
import csv
 
# opening the csv file by specifying
# the location
# with the variable name as csv_file
with open('book_dataset.csv') as csv_file:
 
    # creating an object of csv reader
    # with the delimiter as ,
    csv_reader = csv.reader(csv_file, delimiter = ',')
 
    # list to store the names of columns
    list_of_column_names = []
 
    # loop to iterate through the rows of csv
    for row in csv_reader:
 
        # adding the first row
        list_of_column_names.append(row)
 
        # breaking the loop after the
        # first iteration itself
        break
 
# printing the result
print("List of column names : ",
      list_of_column_names[0])

List of column names :  ['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13', 'language_code', 'num_pages', 'ratings_count', 'text_reviews_count', 'publication_date', 'publisher']


In [20]:
import csv
import uuid
import pandas as pd

user_ids = set()
while len(user_ids) < 20000:
    user_ids.add(str(uuid.uuid4()))

# convert the set to dataframe
user_ids_df = pd.DataFrame(list(user_ids), columns=["user_id"])

# write the dataframe to a csv file
user_ids_df.to_csv("user_ids.csv", index=False)



In [21]:
import csv
import uuid

# Column names of the book dataset
column_names = ['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13', 'language_code', 'num_pages', 'ratings_count', 'text_reviews_count', 'publication_date', 'publisher']

# Number of queries to be generated
num_queries = 1000

# List to store queries
queries = []

# Open the book dataset CSV file
with open('book_dataset.csv', 'r') as book_file:
    # Create a CSV reader
    reader = csv.DictReader(book_file, fieldnames=column_names)
    # Iterate through rows of the book dataset
    for i, row in enumerate(reader):
        # Generate a query ID
        query_id = 'Q' + str(i)
        # Create a list to store the conditions
        conditions = []
        # Iterate through the columns of the book dataset
        for column_name in column_names:
            # Skip the first column (bookID)
            if column_name == 'bookID':
                continue
            # Add the condition to the list
            conditions.append(column_name + '=' + row[column_name])
        # Join the conditions with comma separator
        conditions_str = ','.join(conditions)
        # Append the query to the list
        queries.append([query_id, conditions_str])
        # Break the loop after generating the specified number of queries
        if i == num_queries:
            break

# Write the queries to a CSV file
with open('queries.csv', 'w', newline='') as query_file:
    writer = csv.writer(query_file)
    writer.writerows(queries)


In [22]:
import random
import csv
import uuid

# create the set of user IDs
user_ids = set()
while len(user_ids) < 1000:
    user_ids.add(str(uuid.uuid4()))
    
# create the set of query IDs
query_ids = set()
while len(query_ids) < 10000:
    query_ids.add("Q"+str(uuid.uuid4()))

# create the utility matrix
utility_matrix = []

# add the first row (query IDs)
utility_matrix.append(list(query_ids))

# add the rest of the rows (user IDs and random values)
for user_id in user_ids:
    row = [user_id]
    for query_id in query_ids:
        if random.random() < 0.5:
            row.append(random.randint(1,100))
        else:
            row.append('')
    utility_matrix.append(row)

# write the utility matrix to a CSV file
with open('utility_matrix.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(utility_matrix)


In [23]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Read in the utility matrix
df = pd.read_csv('utility_matrix.csv')
df.fillna(0, inplace=True)  # Replace NaN values with 0

cosine_sim = cosine_similarity(df, dense_output=False)

# Define the number of closest neighbors to use
k = 10

# Next, fill in missing values with estimated values using the cosine similarity matrix
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        if pd.isna(df.iloc[i, j]):
            user_ratings = df.iloc[i, :]
            item_ratings = df.iloc[:, j]
            user_ratings = user_ratings[~pd.isna(user_ratings)]
            item_ratings = item_ratings[~pd.isna(item_ratings)]
            # subtract the mean rating of each user and item
            mean_user_rating = user_ratings.mean()
            mean_item_rating = item_ratings.mean()
            user_ratings -= mean_user_rating
            item_ratings -= mean_item_rating
            similar_users = cosine_sim[i][~pd.isna(df.iloc[:, j])]
            similar_items = cosine_sim[j][~pd.isna(df.iloc[i, :])]
            # get the k closest neighbors
            top_k_users = similar_users.argsort()[-k:]
            top_k_items = similar_items.argsort()[-k:]
            similar_users = similar_users[top_k_users]
            similar_items = similar_items[top_k_items]
            item_ratings = item_ratings[top_k_items]
            df.iloc[i, j] = mean_item_rating + mean_user_rating + (sum(similar_users * similar_items * item_ratings) / sum(similar_users * similar_items))

# save the filled matrix to csv
df.to_csv('utility_matrix_content_based.csv', index=False)


In [24]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Create an imputer with strategy 'mean'
mean_imputer = SimpleImputer(strategy='mean')
  
# Fit the imputer on the original data frame
df_original = pd.read_csv('utility_matrix.csv')
df_original = mean_imputer.fit_transform(df_original)

# Read the content utility matrix from the CSV file
df_con = pd.read_csv('utility_matrix_content_based.csv')
df_con = mean_imputer.transform(df_con)

# Calculate RMSE
rmse = mean_squared_error(df_original, df_con, squared=False)
print("RMSE of content solution:", rmse)

# Calculate MAE
mae = mean_absolute_error(df_original, df_con)
print("MAE of content solution:", mae)


RMSE of content solution: 35.69507555697578
MAE of content solution: 25.23807597654161


In [25]:
import pandas as pd

# Read in the utility matrix
df = pd.read_csv('utility_matrix.csv')
df = df.fillna(df.mean())

# Compute the Pearson correlation coefficient between each pair of users
user_corr = df.corr(method='pearson')

# Fill in missing values with estimated values using collaborative filtering with Pearson correlation
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        if pd.isna(df.iloc[i, j]):
            user_ratings = df.iloc[i, :]
            item_ratings = df.iloc[:, j]
            user_ratings = user_ratings[~pd.isna(user_ratings)]
            item_ratings = item_ratings[~pd.isna(item_ratings)]
            similar_users = user_corr[i][~pd.isna(df.iloc[:, j])]
            df.iloc[i, j] = (similar_users * item_ratings).sum() / similar_users.abs().sum()

# save the filled matrix to csv
df.to_csv('utility_matrix_collaborative_based.csv', index=False)


In [26]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Read the collaborative utility matrix from the CSV file
df_colla = pd.read_csv('utility_matrix_collaborative_based.csv')

# Read the original utility matrix
df_original = pd.read_csv('utility_matrix.csv')

# Fill the missing values in the original utility matrix with 0
df_original.fillna(0, inplace=True)

# Calculate RMSE
rmse = mean_squared_error(df_original, df_colla, squared=False)
print("RMSE of collaborative solution:", rmse)

# Calculate MAE
mae = mean_absolute_error(df_original, df_colla)
print("MAE of collaborative solution:", mae)

RMSE of collaborative solution: 35.69507555697578
MAE of collaborative solution: 25.23807597654161


In [27]:
import time

start = time.time()
# Compute the cosine similarity between each pair of items
from sklearn.metrics.pairwise import cosine_similarity
item_sim = cosine_similarity(df.T)

# Use the cosine similarity to estimate missing ratings
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        if pd.isna(df.iloc[i, j]):
            item_ratings = df.iloc[i, :]
            similar_items = item_sim[j]
            df.iloc[i, j] = (similar_items * item_ratings).sum() / similar_items.sum()
# Compute RMSE for both solution
from sklearn.metrics import mean_squared_error

df_collaborative = pd.read_csv('utility_matrix_collaborative_based.csv')
df_content = pd.read_csv('utility_matrix_content_based.csv')

# RMSE for collaborative
rmse_collaborative = mean_squared_error(df, df_collaborative, squared=False)


# RMSE for content
rmse_content = mean_squared_error(df, df_content, squared=False)

# RMSE total
rmse_tot = rmse_collaborative + rmse_content

# Weights
w1 = 1 - rmse_content / rmse_tot
w2 = 1 - rmse_collaborative / rmse_tot

# Hybrid utility matrix
Ut_hybrid = w1 * df_content + w2 * df_collaborative

# Convert the matrix to a DataFrame
df_hybrid = pd.DataFrame(Ut_hybrid)

# Save the DataFrame to a CSV file
df_hybrid.to_csv('utility_matrix_hybrid.csv', index=False)
end = time.time()

print("Time taken: ", end - start)

Time taken:  289.66928720474243


In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Read the hybrid utility matrix from the CSV file
df_hybrid = pd.read_csv('utility_matrix_hybrid.csv')

# Read the original utility matrix
df_original = pd.read_csv('utility_matrix.csv')

# Fill the missing values in the original utility matrix with 0
df_original.fillna(0, inplace=True)

# Calculate RMSE
rmse = mean_squared_error(df_original, df_hybrid, squared=False)
print("RMSE of hybrid solution:", rmse)

# Calculate MAE
mae = mean_absolute_error(df_original, df_hybrid)
print("MAE of hybrid solution:", mae)


RMSE of hybrid solution: 35.69507555697578
MAE of hybrid solution: 25.23807597654161


In [29]:
import pandas as pd

# Read in the utility matrix
df = pd.read_csv('utility_matrix_hybrid.csv')

# Threshold to consider a query as recommended
threshold = 70

# Create an empty dataframe to store the recommended queries
recommendations = pd.DataFrame(columns=['user', 'query'])

# Iterate over each user and find the queries with an estimated rating above the threshold
for user in df.index:
    recommended_queries = df.loc[user][df.loc[user] > threshold].index
    recommendations = recommendations.append(pd.DataFrame({'user': [user]*len(recommended_queries), 'query': recommended_queries}))

# Save the recommended queries to a CSV file
recommendations.to_csv('recommended_queries.csv', index=False)


In [30]:
import pandas as pd

df = pd.read_csv("utility_matrix_hybrid.csv")
memory_usage = df.memory_usage().sum() / 1024**2

print("Memory usage of the data: {:.2f} MB".format(memory_usage))

Memory usage of the data: 76.29 MB


In [31]:
import pandas as pd

# Read in the recommended queries
recommendations = pd.read_csv('recommended_queries.csv')

# User ID for which to print recommendations
user_id = '50248bff-1475-4bca-81c4-f40eb17435f7'

# Filter the recommendations for the specified user ID
user_recommendations = recommendations[recommendations['user'] == user_id]

# Print the recommended queries for the user
print(user_recommendations['query'].values)


[]
